In [1]:
import pandas as pd
import re

# 데이터프레임을 불러옵니다
df = pd.read_csv('C:/Users/Kdw/Desktop/project/yn_house/deleted_data.csv')
filtered_df = df[
    df['product_name'].astype(str).str.contains('파이프', na=False) &
    ~df['product_name'].astype(str).str.contains('각관', na=False)
]
filtered_df

C:\Users\Kdw\AppData\Local\Temp\ipykernel_1704\2162584299.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/Kdw/Desktop/project/yn_house/deleted_data.csv')


,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,date_converted,TM_FC,TA,SKY,WF,PREP,ST,T,material_usage
4296,2023년 3월 27일,대복리683,재배시설,8mx33m=1 동(264㎡=80평),고정대파이프,20hdx80cm,70,"2,000","140,000",NaN,2023-03-27,2023-03-27 05:00:00,19.0,DB01,맑음,0.0,0.0,1.0,NaN
4297,2023년 10월 18일,터널식하우스,재배시설,8mx25mx1.8m=2 동,고정대파이프,20hdx80cm,100,"2,000","200,000",400㎡=121평,2023-10-18,2023-10-18 05:00:00,7.0,DB01,맑음,0.0,0.0,1.0,NaN
4298,2023년 10월 18일,터널식하우스,재배시설,8mx25mx1.8m=2 동(이중),고정대파이프,20hdx80cm,100,"2,000","200,000",400㎡=121평,2023-10-18,2023-10-18 05:00:00,7.0,DB01,맑음,0.0,0.0,1.0,NaN
4299,2022년 9월 23일,다겹권치보온시설,재배시설,8mx30mx2.5m= 동(240㎡=73평),고정대파이프,20hdx900mm,60,"2,000","120,000",NaN,2022-09-23,2022-09-23 05:00:00,17.0,DB04,흐림,0.0,30.0,1.0,NaN
4300,2021년 11월 30일,터널식하우스,대추비가림시설,8.2mx48m=1 동(394㎡=119평),고정대파이프,20hdx80cm,100,"2,000","200,000",NaN,2021-11-30,2021-11-30 05:00:00,-1.0,DB01,맑음,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87144,2021년 12월 9일,다겹권치보온시설,재배시설,7mx10mx2m (70㎡=21평),파이프고정대,20hdx900mm,30,"2,000","60,000",NaN,2021-12-09,2021-12-09 05:00:00,17.0,DB03,구름많음,0.0,20.0,1.0,NaN
87145,2021년 11월 12일,터널식하우스,재배시설,7mx42mx1.8m,파이프고정대,20hdx800mm,70,"2,000","140,000",294㎡=89평,2021-11-12,2021-11-12 05:00:00,-99.0,DB01,맑음,0.0,0.0,1.0,NaN
87146,2022년 2월 4일,다겹권치보온시설,NaN,8mx33mx2m(264 ㎡=80평),파이프고정대,20hdx800mm,70,"2,000","140,000",NaN,2022-02-04,2022-02-04 05:00:00,5.0,DB01,맑음,0.0,10.0,1.0,NaN
87147,2022년 6월 1일,다겹권치보온시설,재배시설,6mx15mx2m(90㎡=27평),파이프고정대,20hdx900mm,50,"2,000","100,000",NaN,2022-06-01,2022-06-01 05:00:00,16.0,DB01,맑음,0.0,0.0,1.0,NaN


## 공사유형

In [2]:
def split_construction_scale(scale):
    if pd.isna(scale):
        return None, None, None
    
    scale = str(scale)  # 문자열로 변환
    patterns = [
        re.compile(r'(?P<width>\d*\.?\d+)x(?P<length>\d*\.?\d+)x(?P<height>\d*\.?\d+)m'),  # x를 구분자로 사용하는 패턴
        re.compile(r'(?P<width>\d*\.?\d+)mx(?P<length>\d*\.?\d+)mx(?P<height>\d*\.?\d+)m'),  # 기본 패턴
        re.compile(r'폭(?P<width>\d*\.?\d+)mx길이(?P<length>\d*\.?\d+)mx측고(?P<height>\d*\.?\d+)m'),  # 한글 패턴
        re.compile(r'폭(?P<width>\d*\.?\d+)mx측고(?P<height>\d*\.?\d+)m'),  # 폭과 측고만 있는 패턴
        re.compile(r'폭(?P<width>\d*\.?\d+)mx길이(?P<length>\d*\.?\d+)m'),  # 폭과 길이만 있는 한글 패턴
        re.compile(r'(?P<width>\d*\.?\d+)mx(?P<length>\d*\.?\d+)m'),  # 폭과 길이만 있는 경우
        re.compile(r'길이(?P<length>\d*\.?\d+)mx높이(?P<height>\d*\.?\d+)m'),  # 길이와 높이만 있는 경우
        re.compile(r'폭(?P<width>\d*\.?\d+)mx길이(?P<length>\d*\.?\d+)m측고(?P<height>\d*\.?\d+)m'),  # 폭, 길이와 측고가 있는 경우
        re.compile(r'길이(?P<length>\d*\.?\d+)mx폭(?P<width>\d*\.?\d+(?:,\d*\.?\d+)*)mx높이(?P<height>\d*\.?\d+)m'),  # 길이, 폭과 높이 여러 값 포함한 패턴
        re.compile(r'길이(?P<length>\d*\.?\d+)mx폭(?P<width>\d*\.?\d+)m'),  # 길이와 폭만 있는 경우
        re.compile(r'(?P<width>\d*\.?\d+)m폭x(?P<length>\d*\.?\d+)m길이x(?P<height>\d*\.?\d+)m높이'),  # 9.5m폭x11m길이x3.5m높이 패턴
        re.compile(r'폭(?P<width>\d*\.?\d+)x길이(?P<length>\d*\.?\d+)mx측고(?P<height>\d*\.?\d+)m'),  # 폭7.5x길이12mx측고3m
        re.compile(r'폭(?P<width>\d*\.?\d+)x길이(?P<length>\d*\.?\d+)x측고(?P<height>\d*\.?\d+)m'),  # 폭5x길이10x측고1.8m
        re.compile(r'폭(?P<width>\d*\.?\d+)mx길이(?P<lengths>[\d*\.?\d+,]+)m측고(?P<height>\d*\.?\d+)m'),  # 폭8mx길이34,22mx측고3m
        re.compile(r'폭(?P<width>\d*\.?\d+)m,?(?P<lengths>[\d*\.?\d+,]+)m'),  # 18m폭x147m길이 / 36m,51m,17m
        re.compile(r'측고(?P<height>\d*\.?\d+)m\*길이(?P<length>\d*\.?\d+)m'),  # 측고5m*길이400m
        re.compile(r'(?P<width>\d*\.?\d+)mx(?P<length>\d*\.?\d+)x(?P<height>\d*\.?\d+)m'),  # 6mx25x2.3m
        re.compile(r'(?P<width>\d*\.?\d+)m폭x측고(?P<height>\d*\.?\d+)m'),  # 폭8.5mx측고3.6m
    ]
    for pattern in patterns:
        match = pattern.search(scale)
        if match:
            width = match.groupdict().get('width', '0')
            length = match.groupdict().get('length', '0')
            height = match.groupdict().get('height', '0')
            return width, length, height
    
    return None, None, None

# 각 행에 대해 construction_scale 분리 적용
df[['construction_width', 'construction_length', 'construction_height']] = df['construction_scale'].apply(
    lambda x: pd.Series(split_construction_scale(x))
)

# interconnected 열 생성
df['interconnected'] = df['construction_scale'].apply(
    lambda x: 1 if '연동' in str(x) else 0
)

df

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,SKY,WF,PREP,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected
0,2021년 12월 4일,기둥창고,편의시설,8mx10mx2.5m(80 ㎡=24평),(12/27 추가) 개폐기,48x32,38,"1,600","60,800",NaN,...,DB01,맑음,0.0,0.0,1.0,NaN,8,10,2.5,0
1,2021년 1월 28일,영천시,재배시설,7mx76mx3m=5 연동,(12/27 추가) 개폐기,4m interval,100,"2,000","200,000",2660㎡=805평,...,DB03,구름많음,0.0,20.0,1.0,NaN,7,76,3,1
2,2023년 6월 20일,기둥하우스,창고,폭15mx길이14mx측고3m,(12/27 추가) 개폐기,Diastar 0.15 x 4m x 32m,1,"307,000","307,000",210㎡=64평,...,DB03,구름많음,0.0,20.0,1.0,NaN,15,14,3,0
3,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm,8,"5,800","46,400",NaN,...,DB04,흐림,0.0,30.0,1.0,NaN,None,None,None,0
4,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm x 13mm,16,"1,000","16,000",NaN,...,DB04,흐림,0.0,30.0,1.0,NaN,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92349,2020년 8월 2일,천경업,NaN,폭2.2mx길이75mx높이2m,NaN,NaN,NaN,NaN,NaN,NaN,...,DB03,구름많음,0.0,20.0,1.0,NaN,2.2,75,0,0
92350,2021년 3월 3일,터널식창고,농막시설,4mx8mmx2.5m(32 ㎡=10평),NaN,NaN,NaN,NaN,NaN,NaN,...,DB03,구름많음,0.0,20.0,1.0,NaN,4,8,0,0
92351,2021년 3월 23일,터널식하우스,대추비가림,8.2mx48m=2 동(787㎡=238평),NaN,NaN,NaN,NaN,NaN,NaN,...,DB01,맑음,0.0,10.0,1.0,NaN,8.2,48,0,0
92352,2021년 4월 17일,고경면상리리412-1,재배시설,폭37mx길이48mx측고4.5m=5연동,NaN,NaN,NaN,NaN,NaN,1728㎡=522평,...,DB01,맑음,0.0,0.0,1.0,NaN,37,48,4.5,1


## 자재 규격

### 파이프(각관 제외)

In [3]:
import pandas as pd
import re


# 파이프 규격을 분석하여 분리하는 함수 정의
def parse_pipe_spec(spec):
    # 정규표현식 패턴 정의
    patterns = [
        re.compile(r'(?P<diameter>\d*\.?\d+)x(?P<thickness>\d*\.?\d+)tx(?P<length>\d*\.?\d+)m'),
        re.compile(r'(?P<diameter>\d*\.?\d+)x(?P<thickness>\d*\.?\d+)x(?P<length>\d*\.?\d+)m'),
        re.compile(r'(?P<diameter>\d*\.?\d+)(?:mm|m)?x(?P<thickness>\d*\.?\d+)(?:mm|m)?x(?P<length>\d*\.?\d+)m'),
        re.compile(r'(?P<diameter>\d*\.?\d+)(?:mm|m)?x(?P<length>\d*\.?\d+)mm'),
        re.compile(r'(?P<diameter>\d*\.?\d+)(?:mm|m)?x(?P<length>\d*\.?\d+)m'),
        re.compile(r'(?P<length>\d*\.?\d+)m'),
        re.compile(r'(?P<diameter>\d*\.?\d+)mm')
    ]
    
    spec_dict = {
        'diameter': None,
        'thickness': None,
        'length': None
    }
    
    for pattern in patterns:
        match = pattern.search(spec)  # match -> search로 변경하여 전체 문자열에서 패턴 찾기
        if match:
            spec_dict.update({k: v for k, v in match.groupdict().items() if v is not None})
            break
            
    return spec_dict

# 파이프 규격을 분리하여 새로운 열로 추가하는 함수
def apply_pipe_spec(row):
    product_name = str(row['product_name'])  # 문자열로 변환
    spec = str(row['specification'])  # 문자열로 변환
    if '파이프' in product_name and '각관' not in product_name and any(char.isdigit() for char in spec):  # 파이프와 숫자가 포함된 경우
        # 불필요한 문자 제거
        cleaned_spec = re.sub(r'[^0-9xmt\.]', '', spec)
        return parse_pipe_spec(cleaned_spec)
    else:
        return {'diameter': None, 'thickness': None, 'length': None}

# 각 행에 대해 파이프 규격 분리 적용
parsed_specs = df.apply(apply_pipe_spec, axis=1)
parsed_df = pd.DataFrame(parsed_specs.tolist())

# 기존 데이터프레임과 결합
result_df = pd.concat([df, parsed_df], axis=1)

# '파이프'가 포함되고 '각관'이 포함되지 않은 행 필터링
filtered_result_df = result_df[
    result_df['product_name'].astype(str).str.contains('파이프', na=False) &
    ~result_df['product_name'].astype(str).str.contains('각관', na=False)
]
filtered_result_df.dropna(subset=['diameter', 'thickness', 'length'], inplace=True)

# 원래 df와 병합 수행
pipe_filtered_df = pd.merge(df, filtered_result_df, how='left', on=df.columns.tolist())

# 결과 출력
pipe_filtered_df



C:\Users\Kdw\AppData\Local\Temp\ipykernel_1704\2501715266.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_result_df.dropna(subset=['diameter', 'thickness', 'length'], inplace=True)


,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length
0,2021년 12월 4일,기둥창고,편의시설,8mx10mx2.5m(80 ㎡=24평),(12/27 추가) 개폐기,48x32,38,"1,600","60,800",NaN,...,0.0,1.0,NaN,8,10,2.5,0,NaN,NaN,NaN
1,2021년 1월 28일,영천시,재배시설,7mx76mx3m=5 연동,(12/27 추가) 개폐기,4m interval,100,"2,000","200,000",2660㎡=805평,...,20.0,1.0,NaN,7,76,3,1,NaN,NaN,NaN
2,2023년 6월 20일,기둥하우스,창고,폭15mx길이14mx측고3m,(12/27 추가) 개폐기,Diastar 0.15 x 4m x 32m,1,"307,000","307,000",210㎡=64평,...,20.0,1.0,NaN,15,14,3,0,NaN,NaN,NaN
3,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm,8,"5,800","46,400",NaN,...,30.0,1.0,NaN,None,None,None,0,NaN,NaN,NaN
4,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm x 13mm,16,"1,000","16,000",NaN,...,30.0,1.0,NaN,None,None,None,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93979,2020년 8월 2일,천경업,NaN,폭2.2mx길이75mx높이2m,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,1.0,NaN,2.2,75,0,0,NaN,NaN,NaN
93980,2021년 3월 3일,터널식창고,농막시설,4mx8mmx2.5m(32 ㎡=10평),NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,1.0,NaN,4,8,0,0,NaN,NaN,NaN
93981,2021년 3월 23일,터널식하우스,대추비가림,8.2mx48m=2 동(787㎡=238평),NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,1.0,NaN,8.2,48,0,0,NaN,NaN,NaN
93982,2021년 4월 17일,고경면상리리412-1,재배시설,폭37mx길이48mx측고4.5m=5연동,NaN,NaN,NaN,NaN,NaN,1728㎡=522평,...,0.0,1.0,NaN,37,48,4.5,1,NaN,NaN,NaN


### 연결봉 / 새들 / 금구

In [5]:
import pandas as pd
import re

# 연결봉 및 새들 전처리 함수 정의
def process_construction_materials(row):
    product_name = row['product_name']
    if pd.notna(product_name) and ('연결봉' in product_name or '새들' in product_name or '금구'):
        spec = row['specification']
        if pd.notna(spec):
            spec = str(spec)  # 문자열로 변환
            
            # Diameter 패턴
            diameter_match = (re.search(r'(?P<diameter>\d+(\.\d+)?)mm', spec) or 
                              re.search(r'(?P<diameter>\d+(\.\d+)?)∅', spec) or 
                              re.search(r'(?P<diameter>\d+(\.\d+)?)x', spec))
            
            # Thickness 패턴
            thickness_match = re.search(r'(?P<thickness>\d+(\.\d+)?)t', spec)
            
            # Diameter와 Thickness 값 추출
            diameter = diameter_match.group('diameter') if diameter_match else None
            thickness = thickness_match.group('thickness') if thickness_match else None
            
            # 반환할 값 결정
            return diameter, thickness
        
    return None, None

# 데이터프레임을 복사하여 사용
construction_materials_df = pipe_filtered_df.copy()

# 연결봉 및 새들 행에 대해 diameter와 thickness 추출 적용
construction_materials_df[['temp_diameter', 'temp_thickness']] = construction_materials_df.apply(
    lambda row: pd.Series(process_construction_materials(row)), axis=1
)

# diameter 열이 없을 경우 생성하고 temp_diameter 값을 직접 할당
if 'diameter' not in construction_materials_df.columns:
    construction_materials_df['diameter'] = construction_materials_df['temp_diameter']
else:
    # 기존 diameter 열에 temp_diameter 값을 직접 할당
    construction_materials_df.loc[construction_materials_df['diameter'].isna(), 'diameter'] = construction_materials_df['temp_diameter']

# thickness 열이 없을 경우 생성하고 temp_thickness 값을 직접 할당
if 'thickness' not in construction_materials_df.columns:
    construction_materials_df['thickness'] = construction_materials_df['temp_thickness']
else:
    # 기존 thickness 열에 temp_thickness 값을 직접 할당
    construction_materials_df.loc[construction_materials_df['thickness'].isna(), 'thickness'] = construction_materials_df['temp_thickness']

# 임시로 생성한 temp_diameter 및 temp_thickness 열 삭제
construction_materials_df.drop(columns=['temp_diameter', 'temp_thickness'], inplace=True)

construction_materials_df

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length
0,2021년 12월 4일,기둥창고,편의시설,8mx10mx2.5m(80 ㎡=24평),(12/27 추가) 개폐기,48x32,38,"1,600","60,800",NaN,...,0.0,1.0,NaN,8,10,2.5,0,48,None,NaN
1,2021년 1월 28일,영천시,재배시설,7mx76mx3m=5 연동,(12/27 추가) 개폐기,4m interval,100,"2,000","200,000",2660㎡=805평,...,20.0,1.0,NaN,7,76,3,1,None,None,NaN
2,2023년 6월 20일,기둥하우스,창고,폭15mx길이14mx측고3m,(12/27 추가) 개폐기,Diastar 0.15 x 4m x 32m,1,"307,000","307,000",210㎡=64평,...,20.0,1.0,NaN,15,14,3,0,None,None,NaN
3,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm,8,"5,800","46,400",NaN,...,30.0,1.0,NaN,None,None,None,0,50,None,NaN
4,2022년 7월 29일,관수시설,점적관수,1003㎡,(12/27 추가) 개폐기,50mm x 13mm,16,"1,000","16,000",NaN,...,30.0,1.0,NaN,None,None,None,0,50,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93979,2020년 8월 2일,천경업,NaN,폭2.2mx길이75mx높이2m,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,1.0,NaN,2.2,75,0,0,None,None,NaN
93980,2021년 3월 3일,터널식창고,농막시설,4mx8mmx2.5m(32 ㎡=10평),NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,1.0,NaN,4,8,0,0,None,None,NaN
93981,2021년 3월 23일,터널식하우스,대추비가림,8.2mx48m=2 동(787㎡=238평),NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,1.0,NaN,8.2,48,0,0,None,None,NaN
93982,2021년 4월 17일,고경면상리리412-1,재배시설,폭37mx길이48mx측고4.5m=5연동,NaN,NaN,NaN,NaN,NaN,1728㎡=522평,...,0.0,1.0,NaN,37,48,4.5,1,None,None,NaN


### 조리개

In [5]:
import pandas as pd
import re

# 조리개 전처리 함수 정의
def process_aperture(row):
    product_name = row['product_name']
    spec = row['specification']
    
    if pd.notna(product_name) and '조리개' in product_name:
        if pd.notna(spec):
            spec = str(spec)  # 문자열로 변환
            
            # 정규표현식 패턴: '25x32', '32x32', '25', '32 x 32' 등을 처리
            match = re.search(r'(?P<dia1>\d+)(?:x| x )(?P<dia2>\d+)', spec)
            if match:
                dia1 = match.group('dia1')
                dia2 = match.group('dia2')
                return dia1, dia2
            
            # 단일 숫자의 경우 처리
            single_match = re.search(r'(?P<dia>\d+)', spec)
            if single_match:
                dia = single_match.group('dia')
                return dia, None
            
    return None, None


# 데이터프레임을 복사하여 사용
aperture_df = construction_materials_df.copy()

# 조리개 행에 대해 aperture_dia1과 aperture_dia2 추출 적용
aperture_df[['aperture_dia1', 'aperture_dia2']] = aperture_df.apply(
    lambda row: pd.Series(process_aperture(row)), axis=1
)

# 결과 출력
aperture_df[aperture_df['product_name'].str.contains('조리개', na=False)]

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2


### 티고정구

In [6]:
import pandas as pd
import re

# 티고정구 전처리 함수 정의
def process_aperture(row):
    product_name = row['product_name']
    spec = row['specification']
    
    if pd.notna(product_name) and '티고정구' in product_name:
        if pd.notna(spec):
            spec = str(spec)  # 문자열로 변환
            
            # 정규표현식 패턴: '25x32', '32x32', '25', '32 x 32' 등을 처리
            match = re.search(r'(?P<dia1>\d+)(?:x| x |∅|mm)(?P<dia2>\d+)?', spec)
            if match:
                dia1 = match.group('dia1')
                dia2 = match.group('dia2') if match.group('dia2') else dia1
                return dia1, dia2
            
            # 단일 숫자의 경우 처리
            single_match = re.search(r'(?P<dia>\d+)', spec)
            if single_match:
                dia = single_match.group('dia')
                return dia, dia
            
    return None, None

# 데이터프레임을 복사하여 사용
t_saddle_df = aperture_df.copy()

# 조리개와 티고정구 행에 대해 aperture_dia1과 aperture_dia2 추출 적용
t_saddle_df[['aperture_dia1', 'aperture_dia2']] = t_saddle_df.apply(
    lambda row: pd.Series(process_aperture(row)), axis=1
)

# 특정 범위 내에 있지 않은 aperture_dia1 값을 처리
valid_dia1_values = {'32', '25', '60', '48', '50', '22'}

def validate_aperture_dia(row):
    if pd.notna(row['aperture_dia1']) and row['product_name'] and '티고정구' in row['product_name']:
        if row['aperture_dia1'] not in valid_dia1_values:
            row['aperture_dia1'] = 0
            row['aperture_dia2'] = 0
            row['specification'] = 0
    return row

t_saddle_df = t_saddle_df.apply(validate_aperture_dia, axis=1)

# 결과 출력
t_saddle_df[t_saddle_df['product_name'].str.contains('티고정구', na=False)]

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2


### 사철

In [7]:
import pandas as pd
import re

# 함수 정의: '사철'과 '코팅' 문자열을 처리하여 length와 is_coated 값을 설정
def process_steel_wire(df):
    # '사철' 문자열이 포함된 행에 대해 length를 2로 설정
    df.loc[df['product_name'].str.contains('사철', na=False), 'length'] = 2
    
    # 'is_coated' 열을 기본값 0으로 생성
    df['is_coated'] = 0
    
    # '사철' 문자열이 포함된 행 중에서 '코팅' 문자열이 포함된 경우 is_coated를 1로 설정
    df.loc[
        df['product_name'].str.contains('사철', na=False) & 
        df['product_name'].str.contains('코팅', na=False), 'is_coated'
    ] = 1
    
    return df

steel_wire_df = t_saddle_df.copy()
# 데이터프레임을 복사하여 사용
steel_wire_df = process_steel_wire(t_saddle_df)

# 결과 출력
steel_wire_df[steel_wire_df['product_name'].str.contains('사철', na=False)]



,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated


In [8]:
steel_wire_df

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated
0,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,鍮꾨땺,肄뷀똿0.1x600x11m,1,"121,000","121,000",NaN,2010-07-16,NaN,...,0.1,600,11,0,None,None,NaN,None,None,0
1,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,?곌껐遊?25mm,5,200,"1,000",NaN,2010-07-16,NaN,NaN,...,None,None,None,0,None,None,NaN,None,None,0
2,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,吏곸꽑泥?20hdx800mm,20,"2,000","40,000",NaN,2010-07-16,NaN,NaN,...,None,None,None,0,None,None,NaN,None,None,0
3,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,湲덇뎄,32mm,18,700,"12,600",NaN,2010-07-16,NaN,...,None,None,None,0,None,None,NaN,None,None,0
4,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,吏곴껐?쇱뒪,8x16,1,"9,000","9,000",NaN,2010-07-16,NaN,...,None,None,None,0,None,None,NaN,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98481,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?고겢由?Color Curing Sheet 300gx5mx40m,2,"220,000","440,000",NaN,2024-04-01,2024-04-01 5:00,10,...,None,None,None,0,None,None,NaN,None,None,0
98482,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?뚯씠??Dark White 0.1x14mx21m,1,"735,000","735,000",NaN,2024-04-01,2024-04-01 5:00,10,...,None,None,None,0,None,None,NaN,None,None,0
98483,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,鍮꾨땺,Dark White 0.1x5mx20m,1,"250,000","250,000",NaN,2024-04-01,2024-04-01 5:00,...,5,20,0,0,None,None,NaN,None,None,0
98484,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?≪옄??寃쎈퉬,500m,1,"55,000","55,000",NaN,2024-04-01,2024-04-01 5:00,...,None,None,None,0,None,None,NaN,None,None,0


### 환풍기 셔터

In [9]:
import pandas as pd
import re
# 환풍기 셔터 전처리 함수 정의
def process_fan_shutter(row):
    product_name = row['product_name']
    spec = row['specification']
    
    if pd.notna(product_name) and '환풍기 셔터' in product_name:
        if pd.notna(spec):
            spec = str(spec)  # 문자열로 변환
            print(f"Processing spec: {spec}")  # 디버깅용 출력
            
            # 정규표현식 패턴: '150x150', '350x350 with Shutter', '40x40' 등을 처리
            match = re.search(r'(?P<length>\d+)(?:x| x )(?P<height>\d+)', spec)
            if match:
                length = match.group('length')
                height = match.group('height')
                print(f"Match found: length={length}, height={height}")  # 디버깅용 출력
                return length, height
            
            # 단일 숫자의 경우 처리
            single_match = re.search(r'^(?P<length>\d+)(?:mm)?$', spec)
            if single_match:
                length = single_match.group('length')
                print(f"Single match found: length={length}")  # 디버깅용 출력
                return length, length
            
    return None, None

# 데이터프레임을 복사하여 사용
fan_shutter_df = steel_wire_df.copy()

# 환풍기 셔터 행에 대해 length와 height 추출 적용
fan_shutter_df[['length', 'height']] = fan_shutter_df.apply(
    lambda row: pd.Series(process_fan_shutter(row)), axis=1
)

# 결과 출력
fan_shutter_df[fan_shutter_df['product_name'].str.contains('환풍기 셔터', na=False)]



,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,...,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated,height


In [10]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
list(fan_shutter_df['product_name'].unique())

['1',
 '200',
 '2,000',
 '18',
 '100',
 '10',
 '112,000',
 '10,000',
 '110,000',
 '5',
 '12,500',
 '30',
 '192,000',
 '25,000',
 '1,140,000',
 '20,000',
 '16,500',
 '100,000',
 '75,000',
 '12,000',
 '17,000',
 '40',
 '250',
 '20',
 '120,000',
 '5,000',
 '176,000',
 '57,000',
 '1,440,000',
 '50',
 '65,000',
 '2',
 '131,000',
 '80,000',
 '23,000',
 '9,000',
 '15',
 '300',
 '70,000',
 '46',
 '4',
 '700,000',
 '2,652,000',
 '40,000',
 '-',
 '2,160,000',
 '132,000',
 '64,000',
 '2,268,000',
 '6',
 '2,880,000',
 '70',
 '60',
 '350',
 '2,736,000',
 '52',
 '140',
 '90',
 '1200',
 '600',
 '360,000',
 '12',
 '9,525,000',
 '1,300,000',
 '3',
 '16,800',
 '19,600',
 '200,000',
 '8,712,000',
 '396',
 '19,400',
 '500,000',
 '112',
 '500',
 '800',
 '700',
 '56',
 '8',
 '175,000',
 '160,000',
 '80',
 '1,800',
 '130',
 '150',
 '1,200',
 '0.1x600x70m',
 '121',
 '6.5?꿿100m',
 '?명룺0.1x80x130m',
 '8x16',
 '1,500',
 '15,000',
 '25?꿿3p',
 '13,000',
 '900',
 '6m',
 '16hdx0.8m',
 '2m',
 '315,000',
 '186,000',
 

In [11]:
fan_shutter_df

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,date_converted,TM_FC,TA,SKY,WF,PREP,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated,height
0,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,鍮꾨땺,肄뷀똿0.1x600x11m,1,"121,000","121,000",NaN,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,600,11,0,None,None,None,None,None,0,None
1,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,?곌껐遊?25mm,5,200,"1,000",NaN,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
2,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,吏곸꽑泥?20hdx800mm,20,"2,000","40,000",NaN,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
3,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,湲덇뎄,32mm,18,700,"12,600",NaN,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
4,2010??7??16???곕꼸?앹갹怨??띾쭑?쒖꽕,6mx10mx2.3m(60 ??18??,吏곴껐?쇱뒪,8x16,1,"9,000","9,000",NaN,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98481,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?고겢由?Color Curing Sheet 300gx5mx40m,2,"220,000","440,000",NaN,2024-04-01,2024-04-01 5:00,10,DB04,?먮━怨??쒕븣 鍮?1.0,60,1,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
98482,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?뚯씠??Dark White 0.1x14mx21m,1,"735,000","735,000",NaN,2024-04-01,2024-04-01 5:00,10,DB04,?먮━怨??쒕븣 鍮?1.0,60,1,NaN,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None
98483,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,鍮꾨땺,Dark White 0.1x5mx20m,1,"250,000","250,000",NaN,2024-04-01,2024-04-01 5:00,10,DB04,?먮━怨??쒕븣 鍮?1.0,60,1.0,NaN,NaN,NaN,NaN,5,20,0,0,None,None,None,None,None,0,None
98484,2024??4??1???곕꼸?앹갹怨??몄쓽?쒖꽕,7mx20mx2.5m(140 ??43??,?≪옄??寃쎈퉬,500m,1,"55,000","55,000",NaN,2024-04-01,2024-04-01 5:00,10,DB04,?먮━怨??쒕븣 鍮?1.0,60,1.0,NaN,NaN,NaN,NaN,None,None,None,0,None,None,None,None,None,0,None


In [12]:
import pandas as pd

# steel_wire_df의 처음 10개 행을 확인
print(steel_wire_df[['product_name', 'specification']].head(10))

# '환풍기 셔터' 문자열이 포함된 행을 확인
print(steel_wire_df[steel_wire_df['product_name'].str.contains('환풍기 셔터', na=False)][['product_name', 'specification']])


  product_name specification
0            1       121,000
1          200         1,000
2        2,000        40,000
3           18           700
4            1         9,000
5          100        12,000
6          200         8,000
7          100        10,000
8           10           250
9          200           200
Empty DataFrame
Columns: [product_name, specification]
Index: []


In [13]:
import pandas as pd
import re

# 예제 데이터
data = {
    'product_name': ['환풍기 셔터', '환풍기 셔터', '환풍기 셔터', '환풍기 셔터', '환풍기 셔터'],
    'specification': ['40x40', '350x350', '150mm', '600x400', '350x350 with Shutter']
}

steel_wire_df = pd.DataFrame(data)

# 환풍기 셔터 전처리 함수 정의
def process_fan_shutter(row):
    product_name = row['product_name']
    spec = row['specification']
    
    if pd.notna(product_name) and '환풍기 셔터' in product_name:
        if pd.notna(spec):
            spec = str(spec)  # 문자열로 변환
            print(f"Processing spec: {spec}")  # 디버깅용 출력
            
            # 정규표현식 패턴: '150x150', '350x350 with Shutter', '40x40' 등을 처리
            match = re.search(r'(?P<length>\d+)(?:x| x )(?P<height>\d+)', spec)
            if match:
                length = match.group('length')
                height = match.group('height')
                print(f"Match found: length={length}, height={height}")  # 디버깅용 출력
                return length, height
            
            # 단일 숫자의 경우 처리
            single_match = re.search(r'^(?P<length>\d+)(?:mm)?$', spec)
            if single_match:
                length = single_match.group('length')
                print(f"Single match found: length={length}")  # 디버깅용 출력
                return length, length
            
    return None, None

# 데이터프레임을 복사하여 사용
fan_shutter_df = steel_wire_df.copy()

# 환풍기 셔터 행에 대해 length와 height 추출 적용
fan_shutter_df[['length', 'height']] = fan_shutter_df.apply(
    lambda row: pd.Series(process_fan_shutter(row)), axis=1
)

# 결과 출력
print(fan_shutter_df[fan_shutter_df['product_name'].str.contains('환풍기 셔터', na=False)])


Processing spec: 40x40
Match found: length=40, height=40
Processing spec: 350x350
Match found: length=350, height=350
Processing spec: 150mm
Single match found: length=150
Processing spec: 600x400
Match found: length=600, height=400
Processing spec: 350x350 with Shutter
Match found: length=350, height=350
  product_name         specification length height
0       환풍기 셔터                 40x40     40     40
1       환풍기 셔터               350x350    350    350
2       환풍기 셔터                 150mm    150    150
3       환풍기 셔터               600x400    600    400
4       환풍기 셔터  350x350 with Shutter    350    350


In [14]:
fan_shutter_df

,product_name,specification,length,height
0,환풍기 셔터,40x40,40,40
1,환풍기 셔터,350x350,350,350
2,환풍기 셔터,150mm,150,150
3,환풍기 셔터,600x400,600,400
4,환풍기 셔터,350x350 with Shutter,350,350


In [15]:
t_saddle_df[t_saddle_df['product_name'].str.contains('씨형강', na=False)]['specification'].unique()


array([], dtype=object)

In [16]:
t_saddle_df[t_saddle_df['product_name'].str.contains('씨형강', na=False)]

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,date_converted,TM_FC,TA,SKY,WF,PREP,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated


In [17]:
fan_shutter_df

,product_name,specification,length,height
0,환풍기 셔터,40x40,40,40
1,환풍기 셔터,350x350,350,350
2,환풍기 셔터,150mm,150,150
3,환풍기 셔터,600x400,600,400
4,환풍기 셔터,350x350 with Shutter,350,350


In [18]:
t_saddle_df[(t_saddle_df['product_name'] == '사철') & (t_saddle_df['length'].isnull())]

,date,construction_type,construction_sub_type,construction_scale,product_name,specification,quantity,unit_price,total_price,parenthesis_info,date_converted,TM_FC,TA,SKY,WF,PREP,ST,T,material_usage,construction_width,construction_length,construction_height,interconnected,diameter,thickness,length,aperture_dia1,aperture_dia2,is_coated


In [19]:
pipe_filtered_df[pipe_filtered_df['product_name'].str.contains('환풍기 셋트', na=False)]['specification'].value_counts()

Series([], Name: count, dtype: int64)

- 직결피스 규격 알아보기

In [20]:
list(pipe_filtered_df['product_name'].unique())

['1',
 '200',
 '2,000',
 '18',
 '100',
 '10',
 '112,000',
 '10,000',
 '110,000',
 '5',
 '12,500',
 '30',
 '192,000',
 '25,000',
 '1,140,000',
 '20,000',
 '16,500',
 '100,000',
 '75,000',
 '12,000',
 '17,000',
 '40',
 '250',
 '20',
 '120,000',
 '5,000',
 '176,000',
 '57,000',
 '1,440,000',
 '50',
 '65,000',
 '2',
 '131,000',
 '80,000',
 '23,000',
 '9,000',
 '15',
 '300',
 '70,000',
 '46',
 '4',
 '700,000',
 '2,652,000',
 '40,000',
 '-',
 '2,160,000',
 '132,000',
 '64,000',
 '2,268,000',
 '6',
 '2,880,000',
 '70',
 '60',
 '350',
 '2,736,000',
 '52',
 '140',
 '90',
 '1200',
 '600',
 '360,000',
 '12',
 '9,525,000',
 '1,300,000',
 '3',
 '16,800',
 '19,600',
 '200,000',
 '8,712,000',
 '396',
 '19,400',
 '500,000',
 '112',
 '500',
 '800',
 '700',
 '56',
 '8',
 '175,000',
 '160,000',
 '80',
 '1,800',
 '130',
 '150',
 '1,200',
 '0.1x600x70m',
 '121',
 '6.5?꿿100m',
 '?명룺0.1x80x130m',
 '8x16',
 '1,500',
 '15,000',
 '25?꿿3p',
 '13,000',
 '900',
 '6m',
 '16hdx0.8m',
 '2m',
 '315,000',
 '186,000',
 

In [21]:
pipe_filtered_df.to_csv('C:/Users/Kdw/Desktop/project/yn_house/data_preprocessing_final.csv', index=False)

OSError: Cannot save file into a non-existent directory: '\home\chg2022\github_yn\YN_House'